# Load Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchsummary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Set Hyper Parameters

In [3]:
hyper_parameters = {
    "SEED":20180724,
    "notebookName":"MovingWindowSequence",
    "nepochs":10000,
    "batch_size":256,
    "learning_rate":0.001,
    "window_size":128,
    "max_patience_count":1000
}

nepochs = hyper_parameters['nepochs']

In [4]:
torch.manual_seed(hyper_parameters["SEED"])

In [5]:
f"./models/{hyper_parameters['notebookName']}"

'./models/MovingWindowSequence'

In [6]:

PATH = Path(f"./models/{hyper_parameters['notebookName']}")
if os.path.isdir(PATH):
    dir_list = os.listdir(PATH)
    num_files = 0
    while True:
        if os.path.isfile(str(PATH / f"{num_files}")):
            print(num_files)
            num_files += 1
        else:
            break
else:
    os.mkdir(PATH)
    num_files = 0
num_files

0

# Set Path

In [7]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")

# Help Functions

# Load Data

In [8]:
df_train_default = pd.read_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))

In [9]:
df_test = pd.read_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))

In [10]:
for col in df_train_default.columns:
    print(col)

collectionName
phoneName
millisSinceGpsEpoch
latDeg
lngDeg
heightAboveWgs84EllipsoidM
phone
timeSinceFirstFixSeconds
hDop
vDop
speedMps
courseDegree
t_latDeg
t_lngDeg
t_heightAboveWgs84EllipsoidM
constellationType
svid
signalType
receivedSvTimeInGpsNanos
xSatPosM
ySatPosM
zSatPosM
xSatVelMps
ySatVelMps
zSatVelMps
satClkBiasM
satClkDriftMps
rawPrM
rawPrUncM
isrbM
ionoDelayM
tropoDelayM
utcTimeMillis
elapsedRealtimeNanos
yawDeg
rollDeg
pitchDeg
utcTimeMillis_Status
SignalCount
SignalIndex
ConstellationType
Svid
CarrierFrequencyHz
Cn0DbHz
AzimuthDegrees
ElevationDegrees
UsedInFix
HasAlmanacData
HasEphemerisData
BasebandCn0DbHz
utcTimeMillis_UncalMag
elapsedRealtimeNanos_UncalMag
UncalMagXMicroT
UncalMagYMicroT
UncalMagZMicroT
BiasXMicroT
BiasYMicroT
BiasZMicroT
utcTimeMillis_UncalAccel
elapsedRealtimeNanos_UncalAccel
UncalAccelXMps2
UncalAccelYMps2
UncalAccelZMps2
BiasXMps2
BiasYMps2
BiasZMps2
utcTimeMillis_UncalGyro
elapsedRealtimeNanos_UncalGyro
UncalGyroXRadPerSec
UncalGyroYRadPerSec
U

# Dataloader

In [11]:
df_train_default['phone'].value_counts()

2021-04-22-US-SJC-1_Pixel4             2890
2021-04-22-US-SJC-1_SamsungS20Ultra    2826
2020-09-04-US-SF-2_Mi8                 2500
2021-04-29-US-SJC-2_SamsungS20Ultra    2370
2020-09-04-US-SF-2_Pixel4              2349
                                       ... 
2021-01-05-US-SVL-2_Pixel4XL           1193
2020-06-05-US-MTV-1_Pixel4XLModded     1123
2021-04-26-US-SVL-1_Mi8                1036
2021-04-26-US-SVL-1_Pixel5             1034
2020-05-14-US-MTV-2_Pixel4XLModded      577
Name: phone, Length: 73, dtype: int64

# Split functions

In [12]:
def CustomTrainValidSplit(df:pd.DataFrame, valid_size):
    phones = df['phone'].unique()
    
    valid_num = int(len(phones) * valid_size)
    train_num = len(phones) - valid_num
    
    indexes = np.array(range(len(phones)))
    indexes = np.random.choice(indexes, len(indexes), replace = False)
    
    df_train = []
    for phone in phones[indexes[:train_num]]:
        df_train.append(df[df['phone'] == phone])
    df_train = pd.concat(df_train).reset_index().drop(columns = 'index')
    
    df_valid = []
    for phone in phones[indexes[train_num:-1]]:
        df_valid.append(df[df['phone'] == phone])
    df_valid = pd.concat(df_valid).reset_index().drop(columns = 'index')
    
    return df_train, df_valid
    
df_train, df_valid = CustomTrainValidSplit(df_train_default.sort_values(by = ['phone', 'millisSinceGpsEpoch'], ignore_index = True), valid_size = 0.1)
df_test = df_test.sort_values(by = ['phone', 'millisSinceGpsEpoch'], ignore_index = True)
print(df_train.shape, df_valid.shape)
    

(118904, 148) (11245, 148)


In [13]:
df_train

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,timeSinceFirstFixSeconds,hDop,vDop,...,GPS_L1,GPS_L5,GAL_E1,GAL_E5A,GLO_G1,BDS_B1I,BDS_B1C,BDS_B2A,QZS_J1,QZS_J5
0,2021-04-22-US-SJC-1,SamsungS20Ultra,1303160577000,37.334587,-121.899431,-4.10,2021-04-22-US-SJC-1_SamsungS20Ultra,327.00,1.8,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2021-04-22-US-SJC-1,SamsungS20Ultra,1303160578000,37.334564,-121.899395,-5.95,2021-04-22-US-SJC-1_SamsungS20Ultra,328.00,1.8,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2021-04-22-US-SJC-1,SamsungS20Ultra,1303160579000,37.334579,-121.899414,-5.03,2021-04-22-US-SJC-1_SamsungS20Ultra,329.00,1.8,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2021-04-22-US-SJC-1,SamsungS20Ultra,1303160580000,37.334573,-121.899406,-4.78,2021-04-22-US-SJC-1_SamsungS20Ultra,330.00,1.8,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2021-04-22-US-SJC-1,SamsungS20Ultra,1303160581000,37.334588,-121.899438,-9.13,2021-04-22-US-SJC-1_SamsungS20Ultra,331.00,1.8,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118899,2020-05-21-US-MTV-2,Pixel4,1274133324434,37.414966,-122.082347,-22.03,2020-05-21-US-MTV-2_Pixel4,2024.43,1.1,0.0,...,0,0,0,0,0,0,0,0,0,0
118900,2020-05-21-US-MTV-2,Pixel4,1274133325434,37.414952,-122.082351,-22.84,2020-05-21-US-MTV-2_Pixel4,2025.43,1.1,0.0,...,0,0,0,0,0,0,0,0,0,0
118901,2020-05-21-US-MTV-2,Pixel4,1274133326434,37.414959,-122.082353,-23.82,2020-05-21-US-MTV-2_Pixel4,2026.43,1.1,0.0,...,0,0,0,0,0,0,0,0,0,0
118902,2020-05-21-US-MTV-2,Pixel4,1274133327434,37.414961,-122.082353,-22.63,2020-05-21-US-MTV-2_Pixel4,2027.43,1.1,0.0,...,0,0,0,0,0,0,0,0,0,0


# Create Data Loader

## Moving Window Function

In [14]:
def MovingWindow(current_index, max_index, window_size, ratio = 0.5):
    left_window = int(window_size * ratio)
    right_window = window_size - left_window
    
    left_index = np.arange(current_index - left_window, current_index)
    right_index = np.arange(current_index, current_index + right_window)
    
    index = np.concatenate([left_index, right_index])
    index[index < 0] = 0
    index[index >= max_index] = max_index-1
    
    return index

## Custom Dataset

In [15]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.DataFrame, 
                 features = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM'], 
                 labels = ['t_latDeg', 't_lngDeg', 't_heightAboveWgs84EllipsoidM'],
                 window_size = 100,
                 train = False,
                 augment = False,
                 crop_ratio = 0.0,
                device = 'cpu'):
        self.df = df
        self.features = features
        self.labels = labels
        self.len = df.shape[0]
        self.window_size = window_size
        self.train = train
        self.augment = augment
        self.crop_ratio = crop_ratio
        self.device = device
        self.phones = df['phone'].unique()
        
        self.df_dict = dict()
        for phone in tqdm(self.phones):
            self.df_dict[phone] = df[df['phone'] == phone]
        
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        origin_idx = idx
        current_sample = self.df.iloc[idx]
        phone = current_sample['phone']
        millisSinceGpsEpoch = current_sample['millisSinceGpsEpoch']
        df = self.df_dict[phone]
        
        start_index = df.index[0]
        
        self.data = df[features].astype(float).values
        if self.train == True:
            self.true = df[labels].astype(float).values
        else:
            self.true = []
        
        window_index = MovingWindow(idx - start_index, df.shape[0], self.window_size)
        data = self.data[window_index, :]
        
            
        indx = [phone, millisSinceGpsEpoch]
        
        # data shape : num_of_features X window_size
        # true shape : num_of_labels X 1
        data = torch.Tensor(data)
        if self.augment:
            crop_size = int(self.crop_ratio * self.window_size * np.random.rand(1))
            crop_start_index = np.random.randint(0, self.window_size - crop_size)
            data[crop_start_index:crop_size, :] = 0
        
        if self.train is False:
            true = []
        else:
            true = self.true[idx - start_index, :]
            true = torch.Tensor(true.astype(float))
        return data, true, indx
    


## Feature Select

In [16]:
features = [
    'latDeg', 
    'lngDeg', 
    'heightAboveWgs84EllipsoidM',
    'dlatDeg_Scaled',
    'dlngDeg_Scaled',
    'dheight_Scaled',
    'xSatPosM_Scaled',
    'ySatPosM_Scaled',
    'zSatPosM_Scaled',
    'xSatVelMps_Scaled',
    'ySatVelMps_Scaled',
    'zSatVelMps_Scaled',
    'UncalGyroXRadPerSec_Scaled',
    'UncalGyroYRadPerSec_Scaled',
    'UncalGyroZRadPerSec_Scaled',
    'DriftXRadPerSec_Scaled',
    'DriftYRadPerSec_Scaled',
    'DriftZRadPerSec_Scaled',
    'UncalAccelXMps2_Scaled',
    'UncalAccelYMps2_Scaled',
    'UncalAccelZMps2_Scaled',
    'BiasXMps2_Scaled',
    'BiasYMps2_Scaled',
    'BiasZMps2_Scaled',
    'UncalMagXMicroT_Scaled',
    'UncalMagYMicroT_Scaled',
    'UncalMagZMicroT_Scaled',
    'BiasXMicroT_Scaled',
    'BiasYMicroT_Scaled',
    'BiasZMicroT_Scaled',
    'yawDeg_Scaled',
    'rollDeg_Scaled',
    'pitchDeg_Scaled',
    'GPS_L1', 
    'GPS_L5', 
    'GAL_E1', 
    'GAL_E5A', 
    'GLO_G1', 
    'BDS_B1I', 
    'BDS_B1C', 
    'BDS_B2A', 
    'QZS_J1', 
    'QZS_J5'
]
status_features = [
]

labels = [
    't_latDeg', 
    't_lngDeg', 
    't_heightAboveWgs84EllipsoidM',
#     'courseDegree',
#     'hDop',
#     'vDop',
#     'speedMps'
         ]

print(df_train[features].shape)
print(df_train[features].describe())


(118904, 43)
              latDeg         lngDeg  heightAboveWgs84EllipsoidM  \
count  118904.000000  118904.000000               118904.000000   
mean       37.435567    -122.154980                   21.783362   
std         0.082466       0.143705                   73.308272   
min        37.322844    -122.472214                -6157.470000   
25%        37.373910    -122.273127                  -27.310000   
50%        37.424290    -122.116144                    1.770000   
75%        37.469032    -122.069911                   56.780000   
max        37.690836    -121.881855                13701.980000   

       dlatDeg_Scaled  dlngDeg_Scaled  dheight_Scaled  xSatPosM_Scaled  \
count   118904.000000   118904.000000   118904.000000    118904.000000   
mean         0.003680       -0.001599       -0.012571         0.012272   
std          1.013329        1.008292        1.002822         0.724084   
min         -3.367212       -2.837517      -84.852009        -1.703958   
25%         -

## Get Datasets

In [17]:
train_data = CustomDataset(df_train, 
                           features = features, labels = labels, 
                           window_size = hyper_parameters['window_size'], 
                           train = True, 
                           augment = True,
                           crop_ratio = 0.1,
                           device = device)
valid_data = CustomDataset(df_valid, 
                           features = features, labels = labels, 
                           window_size = hyper_parameters['window_size'], 
                           train = True, 
                           augment = False,
                           crop_ratio = 0.0,
                           device = device)
test_data = CustomDataset(df_test, 
                        features = features, labels = labels, 
                        window_size = hyper_parameters['window_size'], 
                        train = False, 
                        augment = False,
                        crop_ratio = 0.0,
                        device = device)

  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

## Get Data Loader

In [18]:
train_loader = DataLoader(train_data, batch_size = hyper_parameters['batch_size'], shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = hyper_parameters['batch_size'], shuffle = False)
test_loader = DataLoader(test_data, batch_size = hyper_parameters['batch_size'], shuffle = False)

# Build Model
## Define Loss and Score

In [19]:
def torch_haversine(lat1, lon1, lat2, lon2):
    lat1=lat1 % 360
    lon1=lon1 % 360
    lat2=lat2 % 360
    lon2=lon2 % 360

    lat1, lat2, lon1, lon2 = map(torch.deg2rad, [lat1, lat2, lon1, lon2])

    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)

    a = torch.sin(dlat / 2.0)**2 + torch.cos(lat1) * torch.cos(lat2) * (torch.sin(dlon / 2.0)**2)
    c = 2 * torch.arcsin(a ** 0.5)

    dist = 6_367_000 * c

    return dist

def CustomLoss(predict:torch.Tensor, target:torch.Tensor):
    loss = SmoothL1Loss(predict, target)
    gpsLoss = GpsLoss(predict, target)
    if (loss < 1):
        loss = gpsLoss
    return loss

def SmoothL1Loss(predict:torch.Tensor, target:torch.Tensor):
    
    return nn.SmoothL1Loss()(predict, target)

def GpsLoss(predict:torch.Tensor, target:torch.Tensor):
    dist = torch_haversine(predict[:,0], predict[:,1], target[:,0], target[:,1])

    return dist.mean()

def GpsScore(predict:torch.Tensor, target:torch.Tensor):
    dist = torch_haversine(predict[:,0], predict[:,1], target[:,0], target[:,1])

    return (torch.quantile(dist, 0.5) + torch.quantile(dist, 0.95))/2

## Build Custom Model

In [20]:

class ConvBlock(nn.Module):
    def __init__(self, input_features, features = 128):
        super().__init__()
        
        self.fuse = nn.Conv1d(input_features + features, features, kernel_size = 1)
        self.conv = nn.Conv1d(input_features, features, kernel_size=3, padding = 1)
        self.batch = nn.BatchNorm1d(features)
        self.pool = nn.AvgPool1d(kernel_size = 2)
        
    def forward(self, x):
        skip = x
        
        x = self.conv(x)
        x = F.relu(x)
        x = self.batch(x)
        x = torch.cat([skip, x], axis = 1)
        x = self.fuse(x)
        x = self.pool(x)
        return x
    
class BaseModel(nn.Module):
    def __init__(self, input_size = (100, 3), output_size = 3):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        self.conv1 = ConvBlock(input_size[1], 256)
        self.conv2 = ConvBlock(256, 512)
        
        self.batch_norm = nn.BatchNorm1d(512)
        self.layer_norm = nn.LayerNorm([32, 512])
        
        self.fc1 = nn.Linear(32*512, 1024)
        self.layer_norm1 = nn.LayerNorm(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.layer_norm2 = nn.LayerNorm(512)
        self.fc3 = nn.Linear(512, output_size)
        
        self.drop06 = nn.Dropout(0.6)
        self.drop03 = nn.Dropout(0.3)
        self.drop01 = nn.Dropout(0.1)
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.kaiming_normal_(self.fc3.weight)
        
        
    def forward(self, x):
        input_size = self.input_size 
        output_size = self.output_size
        
        x[:,:,:2] = torch.deg2rad(x[:,:,:2])
        
        
        x = x.transpose(2,1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.transpose(2,1)
        
        x = x.reshape(-1, 32*512)
        
        x = self.drop06(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.layer_norm1(x)
        
        x = self.drop03(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.layer_norm2(x)
        
        x = self.drop01(x)
        x = self.fc3(x)
        
        x[:,:2] = torch.rad2deg(x[:,:2])
        
        return x

## Compile Model

In [21]:
model = BaseModel((hyper_parameters['window_size'], len(features)), len(labels))
model.to(device)
# model.load_state_dict(torch.load("./models/Baseline3/model-4.pth"))

# loss_func = nn.SmoothL1Loss()
loss_func = CustomLoss
optimizer = optim.Adam(model.parameters(), lr = hyper_parameters['learning_rate'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                mode = 'min',
                                                factor = 0.1,
                                                patience = 5,
                                                verbose = True)

# Fit and Validate
## Train

In [22]:
def train(epoch, progress_log):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    for data, targets, _ in progress_log:
        
        data = data.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        outputs = model(data)  # 데이터를 입력하고 출력을 계산
        loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        predict.append(outputs)
        ground.append(targets)
        
    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    gpsloss = GpsLoss(predict, ground)
    gpsscore = GpsScore(predict, ground)
    return loss, gpsloss, gpsscore

## Valid

In [23]:
def valid(progress_log):
    model.eval()  # 신경망을 추론 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
    predict = []
    ground = []
    
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, targets, _ in progress_log:
            
            data = data.to(device)
            targets = targets.to(device)
            
            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
            
            predict.append(outputs)
            ground.append(targets)
            
    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    gpsloss = GpsLoss(predict, ground)
    gpsscore = GpsScore(predict, ground)
    return loss, gpsloss, gpsscore

## Test

In [24]:
def test(dataloader):
    model.eval()  # 신경망을 추론 모드로 전환
    
    output_list = []
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, _, index in tqdm(dataloader):
            data = data.to(device)
            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            df_temp = pd.DataFrame()
            df_temp['phone'] = index[0]
            df_temp['millisSinceGpsEpoch'] = index[1]
            df_temp[['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']] = outputs[:,:3].to('cpu').numpy()
            output_list.append(df_temp)
    
    predicts = pd.concat(output_list)
    return predicts
            

## Fit

In [25]:
train_loss_list = []
train_gloss_list = []
train_score_list = []
valid_loss_list = []
valid_gloss_list = []
valid_score_list = []

patience_count = 0
min_valid_score = np.inf
checkpoint_name = ""

if not os.path.isdir(f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/"):
    os.mkdir(f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/")
    
prog_epoch = tqdm(range(0, hyper_parameters['nepochs']), position = 0, desc = 'EPOCH')
for epoch in prog_epoch:
    print( "-------------------------------------------------------")
    print(f"|EPOCH: {epoch+1}/{nepochs}")
    prog_train = tqdm(train_loader, desc = 'TRAIN', leave = False)
    prog_valid = tqdm(valid_loader, desc = 'VALID', leave = False)

    train_loss, train_gpsloss, train_gpsscore = train(epoch, prog_train)
    valid_loss, valid_gpsloss, valid_gpsscore = valid(prog_valid)
    
    scheduler.step(valid_gpsscore)
    if valid_gpsscore < min_valid_score:
        print(f"|{epoch+1}-th model is checked!, *model-{epoch}-{valid_gpsscore}.pth*")
        min_valid_score= valid_gpsscore
        checkpoint_name = f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/model-{epoch}-{valid_gpsscore}.pth"
        torch.save(model.state_dict(), checkpoint_name)
    else:
        patience_count+=1
        if(patience_count > hyper_parameters['max_patience_count']):
            break
    
    train_loss_list.append(train_loss)
    train_gloss_list.append(train_gpsloss)
    train_score_list.append(train_gpsscore)
    valid_loss_list.append(valid_loss)
    valid_gloss_list.append(valid_gpsloss)
    valid_score_list.append(valid_gpsscore)
    
    print(f"|TRAIN: loss={train_loss:.6f},  gloss={train_gpsloss:.6f},  score={train_gpsscore:.6f}|")
    print(f"|VALID: loss={valid_loss:.6f},  gloss={valid_gpsloss:.6f},  score={valid_gpsscore:.6f}|")


history = dict()
history['train_loss'] = train_loss_list
history['train_gpsloss'] = train_gloss_list
history['train_score'] = train_score_list
history['valid_loss'] = valid_loss_list
history['valid_gpsloss'] = valid_gloss_list
history['valid_score'] = valid_score_list

EPOCH:   0%|          | 0/10000 [00:00<?, ?it/s]

-------------------------------------------------------
|EPOCH: 1/10000


TRAIN:   0%|          | 0/465 [00:00<?, ?it/s]

VALID:   0%|          | 0/44 [00:00<?, ?it/s]

|1-th model is checked!, *model-0-352097.125.pth*
|TRAIN: loss=20.714237,  gloss=1698378.000000,  score=2857907.750000|
|VALID: loss=8.592450,  gloss=303876.812500,  score=352097.125000|
-------------------------------------------------------
|EPOCH: 2/10000


...

VALID:   0%|          | 0/44 [00:00<?, ?it/s]

|TRAIN: loss=4.690338,  gloss=951921.750000,  score=1446894.250000|
|VALID: loss=1.683973,  gloss=161867.359375,  score=207251.609375|
-------------------------------------------------------
|EPOCH: 58/10000


TRAIN:   0%|          | 0/465 [00:00<?, ?it/s]

VALID:   0%|          | 0/44 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Visualization

In [ ]:
plt.figure(figsize = (16,6))
plt.subplot(3,1,1)
plt.plot(history['train_loss'], label = 'train')
plt.plot(history['valid_loss'], label = 'valid')
plt.ylabel('loss')

plt.subplot(3,1,2)
plt.plot(history['train_gpsloss'], label = 'train')
plt.plot(history['valid_gpsloss'], label = 'valid')
plt.ylabel('gpsloss')

plt.subplot(3,1,3)
plt.plot(history['train_score'], label = 'train')
plt.plot(history['valid_score'], label = 'valid')
plt.ylabel('score')

#  Output

In [ ]:
# Load submission sample
submission = pd.read_csv(str(data_dir / "sample_submission.csv"))
print(submission.shape)
submission.head()

In [ ]:
model.load_state_dict(torch.load(checkpoint_name))
torch.save(model.state_dict(), f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/model-{epoch}-{min_valid_score}.pth")

In [ ]:
predict = test(test_loader)
print(predict.shape)
predict.head()

In [ ]:
submission = submission[['phone', 'millisSinceGpsEpoch']].merge(predict[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']]
                                                                , on = ['phone', 'millisSinceGpsEpoch'])
print(submission.shape)
submission.head()

In [ ]:
submission.to_csv(f"./models/{hyper_parameters['notebookName']}/result-{num_files}.csv", index = False)
pd.DataFrame([]).to_csv(PATH / f"{num_files}")